get as many metrics as possible, require neighborhood, # rooms, etc, take random ratings and rating numbers a few times and find a range via the predictions. 

categories are put in alphabetical order, then the first one is dropped. 


In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
%matplotlib inline

july = pd.read_csv('tomslee_airbnb_new_york_1438_2017-07-12.csv')
july = july.drop(['room_id','survey_id', 'host_id', 'country', 'city', 'bathrooms', 'minstay', 'last_modified', 'location', 'name', 'latitude', 'longitude'], 1)
#remove listings we already know are outliers (price greater than 500 and rooms at 50)
july = july[(july.price < 500) & (july.bedrooms < 50) & (july.overall_satisfaction >= 3)]
july = july[july.bedrooms < 50]
july.head()

,room_type,borough,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price,property_type
0,Shared room,Queens,Jackson Heights,6,5.0,4,1.0,85.0,House
1,Shared room,Brooklyn,Cypress Hills,14,4.0,6,1.0,90.0,House
2,Shared room,Brooklyn,Sheepshead Bay,3,4.5,6,1.0,239.0,Apartment
3,Shared room,Manhattan,Hell's Kitchen,35,5.0,2,1.0,165.0,Apartment
7,Shared room,Manhattan,Harlem,8,5.0,2,1.0,110.0,Apartment


In [2]:
july.room_type.unique()

array(['Shared room', 'Entire home/apt', 'Private room'], dtype=object)

In [4]:
july.property_type.value_counts()

Apartment             19814
House                  2372
Loft                    503
Townhouse               410
Condominium             221
Other                   100
Bed & Breakfast          96
Guesthouse               32
Hostel                   15
Villa                    14
Guest suite              13
Dorm                     12
Bungalow                 11
Boutique hotel            8
Timeshare                 6
Boat                      4
Cabin                     3
Camper/RV                 2
Chalet                    2
Serviced apartment        2
In-law                    2
Vacation home             2
Casa particular           1
Earth House               1
Castle                    1
Name: property_type, dtype: int64

In [10]:
prop_dict = {}
for prop_type in july.property_type.unique():
    prop_dict[prop_type] = 'Other'

In [12]:
prop_dict = {}
for prop_type in july.property_type.unique():
    prop_dict[prop_type] = 'Other'

prop_dict['Apartment'] = 'Apartment'
prop_dict['Bed & Breakfast'] = 'Bed & Breakfast'
prop_dict['Serviced Apartment'] = 'Apartment'
prop_dict['Casa particular'] = 'Bed & Breakfast'
prop_dict['Loft'] = 'Loft'
prop_dict['Townhouse'] = 'Townhouse'
prop_dict['Condominium'] = 'Condominium'
prop_dict['House'] = 'House'

july.property_type.replace(prop_dict, inplace=True)

{'Apartment': 'Apartment',
 'Bed & Breakfast': 'Bed & Breakfast',
 'Boat': 'Other',
 'Boutique hotel': 'Other',
 'Bungalow': 'Other',
 'Cabin': 'Other',
 'Camper/RV': 'Other',
 'Casa particular': 'Bed & Breakfast',
 'Castle': 'Other',
 'Chalet': 'Other',
 'Condominium': 'Condominium',
 'Dorm': 'Other',
 'Earth House': 'Other',
 'Guest suite': 'Other',
 'Guesthouse': 'Other',
 'Hostel': 'Other',
 'House': 'House',
 'In-law': 'Other',
 'Loft': 'Loft',
 'Other': 'Other',
 'Serviced Apartment': 'Apartment',
 'Serviced apartment': 'Other',
 'Timeshare': 'Other',
 'Townhouse': 'Townhouse',
 'Vacation home': 'Other',
 'Villa': 'Other'}

In [33]:
#setting up sorted arrays so position is easy to find
properties = np.sort(july.property_type.unique())
neighbs = np.sort(july.neighborhood.unique())
rooms = np.sort(july.room_type.unique())
bors = np.sort(july.borough.unique())

In [22]:
#setting up arrays that will be put together at the end to make X for prediction
property_types = np.zeros((1, len(july.property_type.unique())-1))
neighborhoods = np.zeros((1, len(july.neighborhood.unique())-1))
boroughs = np.zeros((1, len(july.borough.unique())-1))
room_types = np.zeros((1, len(july.room_type.unique())-1))
numerical = np.zeros((1, 4))

In [ ]:
max_reviews = july.reviews.max()

def set_property(prop_in)
    props = np.searchsorted(properties, prop_in)

    if props != 0:
        property_types[props-1] = 1
        
def set_neighborhood(neighb_in)
    neighb = np.searchsorted(neighbs, neighb_in)

    if neighb != 0:
        neighborhoods[neighb-1] = 1
        
def set_borough(borough_in)
    bor = np.searchsorted(bors, borough_in)

    if bor != 0:
        boroughs[bor-1] = 1

def set_room(room_in)
    room = np.searchsorted(rooms, room_in)

    if room != 0:
        room_types[room-1] = 1

def build_numerical(array, beds_in, accom_in, max_reviews):
    array[3] = accom_in #accomodates
    array[2] = beds_in #num of beds
    array[1] = (np.random.rand * 2) + 3 #rating - generate a number between 3 and 5
    array[0] = np.random.random_integers(1, max_reviews) #num of reviews, generate number between 1 and max # of reviews
        
categories = concatenate((numerical, room_types, boroughs, neighborhoods, property_types))